### BMSB Observation Data

#### 0. Download MN Shapefile

In [25]:
import requests
import zipfile
import os
import arcpy

# Base URL for Minnesota GIS data
MN_BOUNDARY_URL = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_state_of_minnesota/fgdb_bdry_state_of_minnesota.zip"
FOLDER_PATH = "GIS5572"
DESKTOP_PATH = os.path.join(os.path.expanduser("~"), "Desktop")
DATASET_DIRECTORY = os.path.join(DESKTOP_PATH, FOLDER_PATH)

# Function to Retrieve MN Boundary
def download_mn_boundary():
    SAVE_DIR = os.path.join(DATASET_DIRECTORY, "MN_Boundary")
    os.makedirs(SAVE_DIR, exist_ok=True)

    file_name = os.path.join(SAVE_DIR, "mn_boundary.zip")

    print(f"Downloading Minnesota Boundary from: {MN_BOUNDARY_URL}")

    # Download the shapefile zip
    with requests.get(MN_BOUNDARY_URL, stream=True) as r:
        r.raise_for_status()
        with open(file_name, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    print(f"Downloaded Minnesota Boundary: {file_name}")

    # Extract the ZIP file
    with zipfile.ZipFile(file_name, "r") as zip_ref:
        zip_ref.extractall(SAVE_DIR)
    print(f"Extracted Minnesota Boundary to: {SAVE_DIR}")

# Call function to download the MN boundary shapefile
download_mn_boundary()

Downloaded Minnesota Boundary: C:\Users\ethan\Desktop\GIS5572\MN_Boundary\mn_boundary.zip
Extracted Minnesota Boundary to: C:\Users\ethan\Desktop\GIS5572\MN_Boundary


### 1a. Retrieve BSMB Observation Data from iNaturalist

In [ ]:
# iNaturalist API base URL
BASE_URL = "https://api.inaturalist.org/v1/observations"

# Query parameters (Use bounding box for Minnesota)
params = {
    "taxon_id": 81923,      # Brown Marmorated Stink Bug
    "verifiable": "true",   # Only verifiable observations
    "per_page": 200,        # Max 200 results per request (iNaturalist's max)
    "swlat": 43.499,        # Southwest latitude (MN)
    "swlng": -97.2392,      # Southwest longitude (MN)
    "nelat": 49.3843,       # Northeast latitude (MN)
    "nelng": -89.4917       # Northeast longitude (MN)
}

# Initialize a list to store all observations
all_observations = []
page = 1  # Start with the first page

while True:
    # Update the page number in the parameters
    params["page"] = page
    
    # Send GET request to iNaturalist API
    response = requests.get(BASE_URL, params=params)
    data = response.json()

    # Check if data retrieval was successful
    if "results" in data:
        # If no more results, stop the loop
        if not data["results"]:
            break
        
        # Process the observations
        for obs in data["results"]:
            lat = obs.get("geojson", {}).get("coordinates", [None, None])[1]
            lon = obs.get("geojson", {}).get("coordinates", [None, None])[0]

            all_observations.append({
                "ID": obs["id"],
                "Observed Date": obs.get("observed_on", "N/A"),
                "Latitude": lat,
                "Longitude": lon,
                "Scientific Name": obs["taxon"]["name"] if "taxon" in obs else "N/A",
                "Common Name": obs["taxon"].get("preferred_common_name", "N/A"),
                "Image URL": obs["photos"][0]["url"] if obs.get("photos") else None,
                "Location Description": obs.get("place_guess", "Unknown"),
                "Exact Location?": obs.get("location_is_exact", False),
                "Obscured?": obs.get("obscured", False),
                "Accuracy (meters)": obs.get("positional_accuracy", "N/A"),
                "Count Observed": obs.get("individual_count", "N/A")
            })
        
        # Increment the page number for the next request
        page += 1
    else:
        print("Failed to fetch observation data.")
        break

# Convert to Pandas DataFrame
df = pd.DataFrame(all_observations)

# Save to CSV
csv_filename = "brown_marmorated_stink_bug_mn_all_observations.csv"
df.to_csv(csv_filename, index=False)

print(f"Saved {len(df)} observations to {csv_filename}")

# Display the first few rows
print(df.head())

#### 1a. Retrieve MN BMSB Survey Data, Modify for Presence/Absence

In [53]:
from datetime import datetime
import calendar
import requests
import arcpy
import zipfile
import os

# Base URL for Minnesota GIS data
BASE_URL = "https://gisdata.mn.gov/api/3/action/"

FOLDER_PATH = "GIS5572"
DESKTOP_PATH = os.path.join(os.path.expanduser("~"), "Desktop")
DATASET_DIRECTORY = os.path.join(DESKTOP_PATH, FOLDER_PATH)
MN_BOUNDARY = r"C:\Users\ethan\Desktop\GIS5572\MN_Boundary\bdry_state_of_minnesota.gdb\state_of_minnesota"

# FUNCTION TO PULL DATA FROM MNGIS STORE IT IN THE HOME FOLDER
def pullFromMNGIS(datasetName: str):
    SAVE_DIR = os.path.join(DESKTOP_PATH, FOLDER_PATH, datasetName)
    os.makedirs(SAVE_DIR, exist_ok=True)

    # Fetch dataset details
    response = requests.get(f"{BASE_URL}package_show", params={"id": datasetName})
    data = response.json()

    if data["success"]:
        resources = data["result"]["resources"]

        # Find the TIFF or ZIP resource
        tiff_url = None
        for resource in resources:
            if "tif" in resource["url"].lower() or "zip" in resource["url"].lower():
                tiff_url = resource["url"]
                file_name = os.path.join(SAVE_DIR, os.path.basename(tiff_url))
                print(f"Downloading: {tiff_url}")

                # Download the file
                with requests.get(tiff_url, stream=True) as r:
                    r.raise_for_status()
                    with open(file_name, "wb") as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)
                print(f"Downloaded: {file_name}")

                # Unzip if it's a ZIP file
                if file_name.endswith(".zip"):
                    with zipfile.ZipFile(file_name, "r") as zip_ref:
                        zip_ref.extractall(SAVE_DIR)
                    print(f"Extracted: {SAVE_DIR}")

            else:
                print("Failed to fetch dataset information.")

pullFromMNGIS("biota-bmsb")

# Path to your GeoPackage DATASET
BMSB_DATASET_PATH = os.path.join(DATASET_DIRECTORY, "biota-bmsb")
gpkg_path = os.path.join(BMSB_DATASET_PATH, "BMSBSurveyDataTable.dbf")

BMSB_OUTPUT_PATH = os.path.join(DATASET_DIRECTORY, "OUTPUT", "bmsb_mndoag.csv")

aprx = arcpy.mp.ArcGISProject("CURRENT")
map_obj = aprx.listMaps()[0]  # Assuming you want to add to the first map

map_obj.addDataFromPath(gpkg_path)

# CONVERT TO XY POINTS
arcpy.management.XYTableToPoint(
    in_table="BMSBSurveyDataTable",
    out_feature_class=r"BMSBSurveyData_XY_Points",
    x_field="Longitude",
    y_field="Latitude",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

# ADDS CTU DATA TO XY POINTS
arcpy.analysis.SpatialJoin(
    target_features="BMSBSurveyData_XY_Points",
    join_features="ctus_with_id",
    out_feature_class=r"BMSBSurveyData_SpatialJoin",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='SiteName "SiteName" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,SiteName,0,254;SiteType "SiteType" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,SiteType,0,254;City "City" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,City,0,254;County "County" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,County,0,254;Latitude "Latitude" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Latitude,-1,-1;Longitude "Longitude" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Longitude,-1,-1;SurveyName "SurveyName" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,SurveyName,0,254;Scientific "Scientific" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Scientific,0,254;CommonName "CommonName" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,CommonName,0,254;Surveyor "Surveyor" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Surveyor,0,254;Year "Year" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Year,-1,-1;TrapID "TrapID" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,TrapID,0,254;TrapType "TrapType" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,TrapType,0,254;Lure "Lure" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Lure,0,254;CheckDate "CheckDate" true true false 8 Date 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,CheckDate,-1,-1;Adults "Adults" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Adults,-1,-1;Nymphs "Nymphs" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Nymphs,-1,-1;Notes "Notes" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Notes,0,254;GNIS_FEATU "GNIS_FEATU" true true false 10 Long 0 10,First,#,ctus_with_id,GNIS_FEATU,-1,-1;FEATURE_NA "FEATURE_NA" true true false 254 Text 0 0,First,#,ctus_with_id,FEATURE_NA,0,254;CTU_CLASS "CTU_CLASS" true true false 25 Text 0 0,First,#,ctus_with_id,CTU_CLASS,0,25;COUNTY_GNI "COUNTY_GNI" true true false 10 Long 0 10,First,#,ctus_with_id,COUNTY_GNI,-1,-1;COUNTY_COD "COUNTY_COD" true true false 2 Text 0 0,First,#,ctus_with_id,COUNTY_COD,0,2;COUNTY_NAM "COUNTY_NAM" true true false 100 Text 0 0,First,#,ctus_with_id,COUNTY_NAM,0,100;POPULATION "POPULATION" true true false 10 Long 0 10,First,#,ctus_with_id,POPULATION,-1,-1;SHAPE_Leng "SHAPE_Leng" true true false 19 Double 0 0,First,#,ctus_with_id,SHAPE_Leng,-1,-1;SHAPE_Area "SHAPE_Area" true true false 19 Double 0 0,First,#,ctus_with_id,SHAPE_Area,-1,-1;ORIG_FID "ORIG_FID" true true false 10 Long 0 10,First,#,ctus_with_id,ORIG_FID,-1,-1;UNIQUE_ID "UNIQUE_ID" true true false 10 Long 0 10,First,#,ctus_with_id,UNIQUE_ID,-1,-1',
    match_option="WITHIN",
    search_radius=None,
    distance_field_name=""
)

# CLIPS OUT ANYTHING OUTSIDE OF MN BOUNDARIES
arcpy.analysis.PairwiseClip(
    in_features="BMSBSurveyData_SpatialJoin",
    clip_features= MN_BOUNDARY,
    out_feature_class=r"BMSBSurveyData_PairwiseClip",
    cluster_tolerance=None
)

# ADDS FIELD FOR PRESENCE OF BMSB
arcpy.management.AddField(
    in_table="BMSBSurveyData_PairwiseClip",
    field_name="Pres_Abs",
    field_type="SHORT",
    field_precision=None,
    field_scale=None,
    field_length=None,
    field_alias="",
    field_is_nullable="NULLABLE",
    field_is_required="NON_REQUIRED",
    field_domain=""
)

# ADDS FIELD TO PUT IN MONTH
arcpy.management.AddField(
    in_table="BMSBSurveyData_PairwiseClip",
    field_name="Month",
    field_type="SHORT",
    field_precision=None,
    field_scale=None,
    field_length=None,
    field_alias="",
    field_is_nullable="NULLABLE",
    field_is_required="NON_REQUIRED",
    field_domain=""
)

# Iterate through and clean rows
with arcpy.da.UpdateCursor(r"BMSBSurveyData_PairwiseClip", ['Year', 'SiteType', 'Latitude', 'Longitude', 'Adults', 'Nymphs', 'Notes', 'CheckDate','Pres_Abs','Month']) as cursor:  # Use OBJECTID or another unique identifier
    # Iterate through the rows
    for row in cursor:
        # Replace this condition with your logic to identify rows to delete
        if ( not ((row[0] <= 2025) and (row[0] >= 2000) and (row[4] >= 0) and (row[4] <= 1000000) and (row[5] >= 0) and (row[5] <= 1000000) and (row[6] != """Gate Locked""") and (row[6] != """Gate Locked - didn't check"""))):
            # BAD ROW. QAQC BAD DATA REMOVAL
            cursor.deleteRow()   
        else:
            # SETS MONTH AND BMSB PRESENCE
            cursor.updateRow(row[:8]+[int((row[4]+row[5])>0),row[7].month])

# EXPORT FEATURE CLASS TO CSV
BMSB_OUTPUT_PATH = os.path.join(DATASET_DIRECTORY, "OUTPUT", "bmsb_mndoag.csv")
os.mkdir(os.path.join(DATASET_DIRECTORY, "OUTPUT"))
arcpy.conversion.ExportTable(r"BMSBSurveyData_PairwiseClip", BMSB_OUTPUT_PATH)

Downloading: https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_mda/biota_bmsb/shp_biota_bmsb.zip
Downloaded: C:\Users\ethan\Desktop\GIS5572\biota-bmsb\shp_biota_bmsb.zip
Extracted: C:\Users\ethan\Desktop\GIS5572\biota-bmsb
Downloading: https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_mda/biota_bmsb/fgdb_biota_bmsb.zip
Downloaded: C:\Users\ethan\Desktop\GIS5572\biota-bmsb\fgdb_biota_bmsb.zip
Extracted: C:\Users\ethan\Desktop\GIS5572\biota-bmsb
Downloading: https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_mda/biota_bmsb/gpkg_biota_bmsb.zip
Downloaded: C:\Users\ethan\Desktop\GIS5572\biota-bmsb\gpkg_biota_bmsb.zip
Extracted: C:\Users\ethan\Desktop\GIS5572\biota-bmsb
Failed to fetch dataset information.


<class 'FileExistsError'>: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\ethan\\Desktop\\GIS5572\\OUTPUT'

#### 1b. Retrieve iNaturalist BMSB Observations, Modify for Presence/Absence

In [74]:
from datetime import datetime
import requests
import time
import os
import urllib3
import arcpy
import json
import pandas as pd

# --- CONFIGURATION ---
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

API_TOKEN = "YOUR_API_TOKEN_HERE"  # Replace with your real token
BASE_URL = "https://api.inaturalist.org/v1/observations"

FOLDER_NAME = "GIS5572"
DESKTOP_PATH = os.path.join(os.path.expanduser("~"), "Desktop")
DATASET_DIR = os.path.join(DESKTOP_PATH, FOLDER_NAME)
OUTPUT_DIR = os.path.join(DATASET_DIR, "OUTPUT")
os.makedirs(OUTPUT_DIR, exist_ok=True)

CSV_PATH = os.path.join(OUTPUT_DIR, "bmsb_inat_test.csv")
FINAL_OUTPUT = os.path.join(OUTPUT_DIR, "bmsb_inat_final.shp")

XY_FEATURE = os.path.join("memory", "BMSB_inat_XY")
SPATIAL_JOIN_OUTPUT = os.path.join("memory", "BMSB_inat_SpatialJoin")
CLIPPED_OUTPUT = os.path.join("memory", "BMSB_inat_Clip")

MN_BOUNDARY = r"C:\Users\ethan\Desktop\GIS5572\MN_Boundary\bdry_state_of_minnesota.gdb\state_of_minnesota"
CTU_LAYER = "ctus_with_id"  # This should be available in the current map or workspace


# --- Step 1: Pull data from iNaturalist ---
def pull_inaturalist_data():
    headers = {"Authorization": f"Bearer {API_TOKEN}"}
    params = {
        "taxon_id": 81923,  # BMSB
        "verifiable": "true",
        "per_page": 200,
        "swlat": 43.499,
        "swlng": -97.2392,
        "nelat": 49.3843,
        "nelng": -89.4917
    }

    page = 1
    all_obs = []
    print("Downloading iNaturalist data...")

    while True:
        params["page"] = page
        r = requests.get(BASE_URL, params=params, headers=headers, verify=False)
        if r.status_code != 200:
            print(f"Error {r.status_code}: {r.text}")
            break

        results = r.json().get("results", [])
        if not results:
            break

        for obs in results:
            if obs.get("species_guess") == "Brown Marmorated Stink Bug":
                geojson = obs.get("geojson")
                if geojson:
                    coordinates = geojson.get("coordinates", [])
                    if coordinates:
                        date_str = obs.get("observed_on")
                        try:
                            parsed_date = datetime.strptime(date_str, "%Y-%m-%d") if date_str else None
                            month = parsed_date.month if parsed_date else None
                            year = parsed_date.year if parsed_date else None
                        except Exception:
                            parsed_date = None
                            month = None
                            year = None

                        all_obs.append({
                            "Observation ID": obs.get("id"),
                            "Species": obs.get("species_guess"),
                            "Latitude": coordinates[1],
                            "Longitude": coordinates[0],
                            "Date": date_str,
                            "Month": month,
                            "Year": year,
                            "Pres_Abs": 1,
                            "GeoJSON": json.dumps(geojson)
                        })

        print(f"Retrieved page {page} with {len(results)} records.")
        page += 1
        time.sleep(1)

    print(f"Downloaded {len(all_obs)} total observations.")
    return all_obs


# --- Step 2: Process and Save to CSV ---
obs_list = pull_inaturalist_data()
df = pd.DataFrame(obs_list)
df["Latitude"] = pd.to_numeric(df["Latitude"], errors='coerce')
df["Longitude"] = pd.to_numeric(df["Longitude"], errors='coerce')
df.to_csv(CSV_PATH, index=False)
print(f"Saved observations to CSV at: {CSV_PATH}")

# --- Step 3: Convert Table to Points ---
# Convert the Table to Points, for visual check of data
coordinate_system = arcpy.SpatialReference(4326)

arcpy.management.XYTableToPoint(
    in_table=r"C:\Users\ethan\Desktop\GIS5572\OUTPUT\bmsb_inat_test.csv",
    out_feature_class=r"C:\Users\ethan\Documents\ArcGIS\Projects\BMSB_Cleaning\BMSB_Cleaning.gdb\bmsb_inat_XYTableToPoint",
    x_field="Longitude",
    y_field="Latitude",
    coordinate_system=coordinate_system
)

print("CSV table successfully converted to point layer!")

# --- Step 4: Clip Points to MN Boundary ---
arcpy.analysis.PairwiseClip(
    in_features="bmsb_inat_XYTableToPoint",
    clip_features=r"C:\Users\ethan\Desktop\GIS5572\MN_Boundary\bdry_state_of_minnesota.gdb\state_of_minnesota",
    out_feature_class=r"C:\Users\ethan\Documents\ArcGIS\Projects\BMSB_Cleaning\BMSB_Cleaning.gdb\bmsb_inat_PairwiseClip",
    cluster_tolerance=None
)

print("Point layer successfully clipped to MN Boundary!")

# --- Step 5: Spatially Join the Data with CTU Information ---

with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'):
    arcpy.analysis.SpatialJoin(
        target_features="ctus_with_id",
        join_features="bmsb_inat_PairwiseClip",
        out_feature_class=r"C:\Users\ethan\Documents\ArcGIS\Projects\BMSB_Cleaning\BMSB_Cleaning.gdb\bmsb_inat_SpatialJoin",
        join_operation="JOIN_ONE_TO_MANY",
        join_type="KEEP_ALL",
        field_mapping='GNIS_FEATU "GNIS_FEATU" true true false 10 Long 0 10,First,#,ctus_with_id,GNIS_FEATU,-1,-1;FEATURE_NA "FEATURE_NA" true true false 254 Text 0 0,First,#,ctus_with_id,FEATURE_NA,0,253;CTU_CLASS "CTU_CLASS" true true false 25 Text 0 0,First,#,ctus_with_id,CTU_CLASS,0,24;COUNTY_GNI "COUNTY_GNI" true true false 10 Long 0 10,First,#,ctus_with_id,COUNTY_GNI,-1,-1;COUNTY_COD "COUNTY_COD" true true false 2 Text 0 0,First,#,ctus_with_id,COUNTY_COD,0,1;COUNTY_NAM "COUNTY_NAM" true true false 100 Text 0 0,First,#,ctus_with_id,COUNTY_NAM,0,99;POPULATION "POPULATION" true true false 10 Long 0 10,First,#,ctus_with_id,POPULATION,-1,-1;SHAPE_Leng "SHAPE_Leng" true true false 19 Double 0 0,First,#,ctus_with_id,SHAPE_Leng,-1,-1;SHAPE_Area "SHAPE_Area" true true false 19 Double 0 0,First,#,ctus_with_id,SHAPE_Area,-1,-1;ORIG_FID "ORIG_FID" true true false 10 Long 0 10,First,#,ctus_with_id,ORIG_FID,-1,-1;UNIQUE_ID "UNIQUE_ID" true true false 10 Long 0 10,First,#,ctus_with_id,UNIQUE_ID,-1,-1;Observatio "Observation ID" true true false 4 Long 0 0,First,#,bmsb_inat_PairwiseClip,Observation_ID,-1,-1;Species "Species" true true false 8000 Text 0 0,First,#,bmsb_inat_PairwiseClip,Species,0,7999;Latitude "Latitude" true true false 8 Double 0 0,First,#,bmsb_inat_PairwiseClip,Latitude,-1,-1;Longitude "Longitude" true true false 8 Double 0 0,First,#,bmsb_inat_PairwiseClip,Longitude,-1,-1;Date "Date" true true false 8 DateOnly 0 0,First,#,bmsb_inat_PairwiseClip,Date,-1,-1;Month "Month" true true false 4 Long 0 0,First,#,bmsb_inat_PairwiseClip,Month,-1,-1;Year "Year" true true false 4 Long 0 0,First,#,bmsb_inat_PairwiseClip,Year,-1,-1;Pres_Abs "Pres_Abs" true true false 4 Long 0 0,First,#,bmsb_inat_PairwiseClip,Pres_Abs,-1,-1;GeoJSON "GeoJSON" true true false 8000 Text 0 0,First,#,bmsb_inat_PairwiseClip,GeoJSON,0,7999',
        match_option="INTERSECT",
        search_radius=None,
        distance_field_name="",
        match_fields=None
    )

print("Data was successfully joined!")

# --- Step 6: EXPORT FEATURE CLASS TO CSV ---
BMSB_OUTPUT_PATH = os.path.join(OUTPUT_DIR, "bmsb_inat_obs.csv")
os.makedirs(OUTPUT_DIR, exist_ok=True)
arcpy.conversion.ExportTable("bmsb_inat_SpatialJoin", BMSB_OUTPUT_PATH)
print(f"Exported feature class to CSV: {BMSB_OUTPUT_PATH}")

Retrieved page 1 with 200 records.
Retrieved page 2 with 200 records.
Retrieved page 3 with 200 records.
Retrieved page 4 with 200 records.
Retrieved page 5 with 200 records.
Retrieved page 6 with 200 records.
Retrieved page 7 with 95 records.
Downloaded 904 total observations.
Saved observations to CSV at: C:\Users\ethan\Desktop\GIS5572\OUTPUT\bmsb_inat_test.csv


Exported feature class to CSV: C:\Users\ethan\Desktop\GIS5572\OUTPUT\bmsb_inat_obs.csv


### 2. Join BSMB Obsservations and Convert to Arrays
Create an array that shows all month/year combinations for the observation cycle with a 1 for observed and a 0 for not observed.

In [ ]:
import arcpy
import pandas as pd

# Path to your GDB table (not a CSV)
inat_table = r"C:\Users\ethan\Desktop\GIS5572\OUTPUT\bmsb_inat_obs.csv"
doag_table = r"C:\Users\ethan\Desktop\GIS5572\OUTPUT\bmsb_mndoag.csv"

# Step 1: Convert ArcGIS table to pandas DataFrame
fields = ['UNIQUE_ID', 'Month', 'Year', 'Pres_Abs']  # adjust to match your actual field names
data = [row for row in arcpy.da.SearchCursor(table_path, fields)]
df = pd.DataFrame(data, columns=fields)

# Drop or fill rows where 'Month' or 'Year' is NaN
df_cleaned = df.dropna(subset=['Month', 'Year'])

# Convert 'Month' and 'Year' to integers (safe now)
df_cleaned['Month'] = df_cleaned['Month'].astype(int)
df_cleaned['Year'] = df_cleaned['Year'].astype(int)

# Create 'Month_Year' string
df_cleaned['Month_Year'] = df_cleaned['Month'].astype(str).str.zfill(2) + "/" + df_cleaned['Year'].astype(str)

# Step 2: Build full date range and CTU list
date_range = pd.date_range(start="2018-10-01", end="2025-07-01", freq='MS')
date_strs = [f"{d.month:02d}/{d.year}" for d in date_range]
all_ctu_ids = pd.Series([row[0] for row in arcpy.da.SearchCursor(table_path, ['UNIQUE_ID'])]).unique()
ctu_ids = all_ctu_ids

# Step 3: Build complete CTU x month/year grid
all_combos = pd.MultiIndex.from_product(
    [ctu_ids, date_strs], names=['UNIQUE_ID', 'Month_Year']
).to_frame(index=False)

# Step 4: Merge observations with full grid
df_obs = df_cleaned[['UNIQUE_ID', 'Month_Year', 'Pres_Abs']].drop_duplicates()
merged = pd.merge(all_combos, df_obs, on=['UNIQUE_ID', 'Month_Year'], how='left')
merged['Pres_Abs'] = merged['Pres_Abs'].fillna(0)

# Step 5: Pivot to make CTU x Time matrix
pivot = merged.pivot(index='UNIQUE_ID', columns='Month_Year', values='Pres_Abs')

# Step 6: Sort columns by date
pivot = pivot[sorted(pivot.columns, key=lambda x: pd.to_datetime(x, format="%m/%Y"))]

# Step 7: Apply cumulative presence logic
cumulative = pivot.apply(lambda row: row.cumsum().clip(upper=1), axis=1)

# Optional: convert to numpy array
presence_array = cumulative.to_numpy()

# Preview
print("Shape:", presence_array.shape)
print(cumulative.head())

In [83]:
import pandas as pd
import os

# --- File paths ---
inat_table = r"C:\Users\ethan\Desktop\GIS5572\OUTPUT\bmsb_inat_obs.csv"
doag_table = r"C:\Users\ethan\Desktop\GIS5572\OUTPUT\bmsb_mndoag.csv"

# --- Load both datasets ---
inat_df = pd.read_csv(inat_table)
doag_df = pd.read_csv(doag_table)

# --- Clean DOAG: drop nulls and keep earliest observation per UNIQUE_ID ---
doag_df = doag_df.dropna(subset=['UNIQUE_ID', 'Month', 'Year'])
doag_df['Month'] = doag_df['Month'].astype(int)
doag_df['Year'] = doag_df['Year'].astype(int)

# Create a proper date column to find the earliest observation
doag_df['Date'] = pd.to_datetime(doag_df['Year'].astype(str) + '-' + doag_df['Month'].astype(str).str.zfill(2) + '-01')

# Keep only the first record per UNIQUE_ID
doag_earliest = doag_df.sort_values('Date').drop_duplicates(subset='UNIQUE_ID', keep='first')

# Optional: Drop the 'Date' column after filtering
doag_earliest = doag_earliest.drop(columns='Date')

# --- Combine with iNat data ---
inat_df = inat_df.dropna(subset=['UNIQUE_ID', 'Month', 'Year'])
inat_df['Month'] = inat_df['Month'].astype(int)
inat_df['Year'] = inat_df['Year'].astype(int)

# Combine cleaned tables
combined_df = pd.concat([inat_df, doag_earliest], ignore_index=True)

# --- Create Month_Year and clean Pres_Abs ---
combined_df['Month_Year'] = combined_df['Month'].astype(str).str.zfill(2) + "/" + combined_df['Year'].astype(str)
combined_df['Pres_Abs'] = combined_df['Pres_Abs'].fillna(0).astype(int)

# --- Full CTU x Month-Year grid ---
date_range = pd.date_range(start="2018-10-01", end="2025-07-01", freq='MS')
date_strs = [f"{d.month:02d}/{d.year}" for d in date_range]
ctu_ids = combined_df['UNIQUE_ID'].unique()
all_combos = pd.MultiIndex.from_product(
    [ctu_ids, date_strs], names=['UNIQUE_ID', 'Month_Year']
).to_frame(index=False)

# --- Merge with presence data ---
obs_df = combined_df[['UNIQUE_ID', 'Month_Year', 'Pres_Abs']].drop_duplicates()
merged = pd.merge(all_combos, obs_df, on=['UNIQUE_ID', 'Month_Year'], how='left')
merged['Pres_Abs'] = merged['Pres_Abs'].fillna(0)

# --- Pivot to matrix ---
pivot = merged.pivot(index='UNIQUE_ID', columns='Month_Year', values='Pres_Abs')
pivot = pivot[sorted(pivot.columns, key=lambda x: pd.to_datetime(x, format="%m/%Y"))]

# --- Fill from first observation onward ---
def fill_from_first_observation(row):
    obs_idx = row[row == 1].index
    if len(obs_idx) == 0:
        return row * 0
    first_idx = row.index.get_loc(obs_idx[0])
    row.iloc[first_idx:] = 1
    row.iloc[:first_idx] = 0
    return row

cumulative = pivot.apply(fill_from_first_observation, axis=1)

# --- Output path ---
output_csv = os.path.join(os.path.dirname(inat_table), "bmsb_combined_presence_matrix.csv")
cumulative.to_csv(output_csv)

# --- Preview ---
print("Matrix shape:", cumulative.shape)
print(cumulative.head())

Matrix shape: (261, 82)
Month_Year  10/2018  11/2018  12/2018  01/2019  02/2019  03/2019  04/2019  \
UNIQUE_ID                                                                   
1               0.0      0.0      0.0      0.0      0.0      0.0      0.0   
9               0.0      0.0      0.0      0.0      0.0      0.0      0.0   
30              0.0      0.0      0.0      0.0      0.0      0.0      0.0   
32              0.0      0.0      0.0      0.0      0.0      0.0      0.0   
45              0.0      0.0      0.0      0.0      0.0      0.0      0.0   

Month_Year  05/2019  06/2019  07/2019  ...  10/2024  11/2024  12/2024  \
UNIQUE_ID                              ...                              
1               0.0      0.0      0.0  ...      0.0      0.0      0.0   
9               0.0      0.0      0.0  ...      0.0      0.0      0.0   
30              0.0      0.0      0.0  ...      0.0      0.0      0.0   
32              0.0      0.0      0.0  ...      1.0      1.0      1.0  

In [86]:
import arcpy
import pandas as pd
import os

# --- File paths ---
inat_table = r"C:\Users\ethan\Desktop\GIS5572\OUTPUT\bmsb_inat_obs.csv"
doag_table = r"C:\Users\ethan\Desktop\GIS5572\OUTPUT\bmsb_mndoag.csv"
ctu_layer = "ctus_with_id"

# --- Load observation datasets ---
inat_df = pd.read_csv(inat_table)
doag_df = pd.read_csv(doag_table)

# --- Clean DOAG: keep earliest observation per UNIQUE_ID ---
doag_df = doag_df.dropna(subset=['UNIQUE_ID', 'Month', 'Year'])
doag_df['Month'] = doag_df['Month'].astype(int)
doag_df['Year'] = doag_df['Year'].astype(int)
doag_df['Date'] = pd.to_datetime(doag_df['Year'].astype(str) + '-' + doag_df['Month'].astype(str).str.zfill(2) + '-01')
doag_earliest = doag_df.sort_values('Date').drop_duplicates(subset='UNIQUE_ID', keep='first').drop(columns='Date')

# --- Clean iNat ---
inat_df = inat_df.dropna(subset=['UNIQUE_ID', 'Month', 'Year'])
inat_df['Month'] = inat_df['Month'].astype(int)
inat_df['Year'] = inat_df['Year'].astype(int)

# --- Combine both tables ---
combined_df = pd.concat([inat_df, doag_earliest], ignore_index=True)
combined_df['Month_Year'] = combined_df['Month'].astype(str).str.zfill(2) + "/" + combined_df['Year'].astype(str)
combined_df['Pres_Abs'] = combined_df['Pres_Abs'].fillna(0).astype(int)

# --- Get full list of CTUs from master layer ---
ctu_ids = sorted({row[0] for row in arcpy.da.SearchCursor(ctu_layer, ['UNIQUE_ID'])})

# --- Build full CTU x Month-Year grid ---
date_range = pd.date_range(start="2018-10-01", end="2025-07-01", freq='MS')
date_strs = [f"{d.month:02d}/{d.year}" for d in date_range]
all_combos = pd.MultiIndex.from_product([ctu_ids, date_strs], names=['UNIQUE_ID', 'Month_Year']).to_frame(index=False)

# --- Merge with observations ---
obs_df = combined_df[['UNIQUE_ID', 'Month_Year', 'Pres_Abs']].drop_duplicates()
merged = pd.merge(all_combos, obs_df, on=['UNIQUE_ID', 'Month_Year'], how='left')
merged['Pres_Abs'] = merged['Pres_Abs'].fillna(0)

# --- Pivot to matrix ---
pivot = merged.pivot(index='UNIQUE_ID', columns='Month_Year', values='Pres_Abs')
pivot = pivot[sorted(pivot.columns, key=lambda x: pd.to_datetime(x, format="%m/%Y"))]

# --- Apply "1 from first observation forward" logic ---
def fill_from_first(row):
    ones = row[row == 1]
    if ones.empty:
        return row * 0
    first_index = row.index.get_loc(ones.index[0])
    row.iloc[first_index:] = 1
    row.iloc[:first_index] = 0
    return row

cumulative = pivot.apply(fill_from_first, axis=1)

# --- Save cumulative presence matrix to CSV ---
output_csv = os.path.join(os.path.dirname(inat_table), "bmsb_combined_presence_matrix.csv")
cumulative.to_csv(output_csv)

print(f"Saved cumulative presence matrix to: {output_csv}")

# --- Preview ---
print("✅ Matrix created with shape:", cumulative.shape)
print(cumulative.head())

✅ Matrix created with shape: (2743, 82)
Month_Year  10/2018  11/2018  12/2018  01/2019  02/2019  03/2019  04/2019  \
UNIQUE_ID                                                                   
0               0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1               0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2               0.0      0.0      0.0      0.0      0.0      0.0      0.0   
3               0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4               0.0      0.0      0.0      0.0      0.0      0.0      0.0   

Month_Year  05/2019  06/2019  07/2019  ...  10/2024  11/2024  12/2024  \
UNIQUE_ID                              ...                              
0               0.0      0.0      0.0  ...      0.0      0.0      0.0   
1               0.0      0.0      0.0  ...      0.0      0.0      0.0   
2               0.0      0.0      0.0  ...      0.0      0.0      0.0   
3               0.0      0.0      0.0  ...      0.0    